# Import Headers

In [19]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

# Setup LLM

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

response = llm.invoke("Write a SQL query to get top 5 customers by sales. keep it short")
print(response.content)

SELECT customer_id, SUM(sales_amount) as total_sales
FROM sales
GROUP BY customer_id
ORDER BY total_sales DESC
LIMIT 5;


# Connect With Database

In [16]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

# print(db.table_info)

______________

# Ask questions with Zero shot learning

In [17]:
from langchain.agents import create_sql_agent

sql_agent = create_sql_agent(llm, db=db, agent_type="zero-shot-react-description") # set verbose=True to check how it thinks

## Question 1

In [ ]:
q1 = sql_agent.invoke("How many t-shirts do we have left for Nike in extra small size and white color?")
print(q1)

{'input': 'How many t-shirts do we have left for Nike in extra small size and white color?', 'output': '0'}


- ##### `0` is incorrect. The correct answer is `25`.

## Question 2

In [ ]:
q2 = sql_agent.invoke("How much is the price of the inventory for all small size t-shirts?")
print(q2)

{'input': 'How much is the price of the inventory for all small size t-shirts?', 'output': 'The price of the inventory for all small size t-shirts is $13,429.'}


- ##### `0` is incorrect. The correct answer is `25`.

# Question 3

In [ ]:
q3 = sql_agent.invoke("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")
print(f"Input:\n{q3['input']}\nOutput:\n{q3['output']}")

- ##### `0` is incorrect. The correct answer is `25`.

# Question 4

In [ ]:
q4 = sql_agent.invoke("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")
print(q4)

# Question 5

In [ ]:
q5 = sql_agent.invoke("How many white color Levi's t shirts we have available?")
print(q5)

_____________________

__________________